In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('Transformation') \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

sc = spark.sparkContext

# Transpormations (define a new RDD)
-  RDD를 변경하는 작업들
-  결과물로 다시 RDD가 생성
-  e.g., map(), flatMap(), sample(), reduceByKey()

In [11]:
data = sc.parallelize(range(1, 11), 1)

## map(func)
- RDD의 각 element에 함수 func를 적용하여 새로운 RDD를 리턴한다.
- 각 element에 하나씩 mapping되는 값을 반환하므로 1:1 함수이다.

In [12]:
data = sc.parallelize(range(1, 11), 1)
data = data.map(lambda x: x + 1)
data.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## filter(func)
- 전달된 함수의 조건에 따라 통과한 값만 출력 -> 반드시 bool 값을 반환

In [13]:
data = sc.parallelize(range(1, 11), 1)
filtered = data.filter(lambda x: x < 5)
filtered.collect()

[1, 2, 3, 4]

## flatMap(func)
- `map()`과 비슷하나 element하나를 받아서 0, 1개 혹은 복수개를 출력할 수 있다. 즉 1:다 함수이다.

In [14]:
data = sc.parallelize([1, 2, 3, 4], 1)
print(data.map(lambda x: [x, x*x]).collect())
print(data.flatMap(lambda x: [x, x*x]).collect())

[[1, 1], [2, 4], [3, 9], [4, 16]]
[1, 1, 2, 4, 3, 9, 4, 16]


## mapPartitions(func)
- RDD의 각 파티션에 대해 map을 실행. 다:1(partition 갯수)
- 활용성은 낮음

In [15]:
data = sc.parallelize(range(1, 11), 3)
def f(x): yield sum(x)
data.mapPartitions(f).collect()

[6, 15, 34]

## mapValues(func)
- 키-값 쌍 RDD에 대해 값에 대해서만 func를 적용

In [16]:
data = sc.parallelize([('a', ['apple', 'banana', 'lemon']), ('b', ['grape'])])
def f(x): return len(x)
data.mapValues(f).collect()

[('a', 3), ('b', 1)]

## reduceByKey(func, [numPartitions])
- key별로 func를 수행
- numPartitions: 결과로 생성되는 RDD의 파티션 갯수

In [17]:
data = sc.parallelize([('a', 1), ('b', 2), ('c', 3), ('a', 4)])
data.reduceByKey(lambda x, y: x + y).collect()

[('a', 5), ('b', 2), ('c', 3)]

In [18]:
sc.stop()

# Word Count